# 04 — TaskFlow: Docker Compose, Execução, Healthchecks e Troubleshooting

**Objetivo:** correr **gateway + tasks-service** via **Docker Compose**, com *healthchecks* e guia de resolução de problemas.

## `docker-compose.yml` (trecho principal)

```yaml
services:
  tasks-service:
    build: ./tasks-service
    ports: ["8081:8081"]
    healthcheck:
      test: ["CMD", "wget", "-qO-", "http://localhost:8081/actuator/health"]
      interval: 10s
      timeout: 3s
      retries: 8

  gateway:
    build: ./gateway
    ports: ["8080:8080"]
    environment:
      - TASKS_BASE_URL=http://tasks-service:8081
    depends_on:
      tasks-service:
        condition: service_healthy
    healthcheck:
      test: ["CMD", "wget", "-qO-", "http://localhost:8080/actuator/health"]
      interval: 10s
      timeout: 3s
      retries: 8
```


## Arrancar e testar

```bash
./run_compose.sh
# ou
docker compose up --build

# testar via gateway
curl -s http://localhost:8080/api/tasks | jq

# direto ao serviço
curl -s http://localhost:8081/tasks | jq
```


## Troubleshooting (erros reais que vimos)

- **`UnknownHost tasks-service`** (no gateway): o gateway está fora do Docker a apontar para o hostname do Compose.  
  ➜ usa `TASKS_BASE_URL=http://localhost:8081` (local) ou perfis/variável no compose.

- **`no main manifest attribute`** ao iniciar o *container* do gateway: jar não foi *repackaged*.  
  ➜ garante o `spring-boot-maven-plugin` com *goal* `repackage`; no Dockerfile copia o jar **sem** `.original`.

- **404 Whitelabel**: estás a chamar `/api/tasks` no backend direto (8081).  
  ➜ no backend usa `/tasks`; via gateway usa `/api/tasks`.

- **Conflito MVC vs WebFlux**: gateway no classpath do backend.  
  ➜ remove dependências do gateway do `tasks-service`; força `spring.main.web-application-type=servlet` se precisares.


## Próxima etapa (teaser da Fase 2)
- Introduzir **Spring Cloud Config** e **Eureka**, mantendo as rotas e só trocando o `uri` para `lb://tasks-service`.
- Adicionar **Resilience4j** com *timeouts*, *retries* e *circuit breakers*.
